# Model Universes

AST 3414 - Spring 2026

## Introduction

This notebook allows you to create plots of scale-factor vs. time for whatever cosmological recipe you like in a universe that follows the Friedmann Roberston Walker metric.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

While Python does have some native functionality (it is able to add numbers, make lists, etc.), it is often not sufficient for any code that anyone writes. Thus, it is necessary to import some libraries - pieces of code that various people have written to enable more complex functions.

Two of these libraries are going to be used quite extensively, numpy (often abbreviated as np), and matplotlib pyplot (often abbreviated as plt). We will always begin with importing them, and any other libraries that we will need.

I like to import the math library and set plots created by matplotlib to be displayed in the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

Let's create a few commands that show how Python works. We'll start with a print statement - which is incredibly useful whether you just want to see an output of some calculations, or when you are trying to debug the code to understand what it is that you have done. Print statements are your friend, use them frequently.

Anything that is written after a # is considered to be a comment that isn't executed.

In [ ]:
print("The sum of 2+2 is")
print(2+2)
hello="Hello World!"
print("You can even print out multiple things at once - ", 1234, hello)
print("If you want to take a number to some power, you should use ** instead of ^, like this - ", 2**4)
#print(This line will not print, it is commented out with a # in the front)

Often in this class, you will need to create an array of numbers. You can do it in a variety of different ways. All of the three "functions" introduced in the next cell are a part of numpy library, so to call them you need to first reference the library (which we imported as np), followed by the name of the function. A function in Python is similar to the mathematical definition of a function in that it (generally) has some input (which is what is inside the parenthesis of the function) and returns some output.

In [ ]:
arr1 = np.array([1,2,3,4])
arr2 = np.arange(10)
arr3 = np.arange(20,30)
arr4 = np.arange(50,100,10)
arr5 = np.linspace(0,1,10)
print(arr1)
print(arr2)
print(arr3)
print(arr4)
print(arr5)

# Question 1

1. Notice that the arange function can be called with just a single argument that is passed to the function, or with two, or with three. *What* is the difference between these methods?
2. What do these arguments signify?

# ANSWER HERE

Today you will be using a simple integrator to solve Friedmann's equation for the scale factor as a function of time. You will need to assume the values of the dimensionless densities for matter, radiation, and the cosmological constant $\Lambda$.

Conveniently, the `astropy` library has all the latest cosmological parameters available as a module. Here we import them to simulate our universe.

In [ ]:
# Import cosmological constants from latest Planck resutls via astropy
import astropy.units as u
from astropy.constants import G, h, k_B
from astropy.cosmology import Planck15
omega_m = Planck15.Odm0+Planck15.Ob0 # total matter = dark matter + baryons
omega_r = Planck15.Ogamma0
omega_de = Planck15.Ode0
H0 = Planck15.H0.to(1/u.Gyr) # Hubble's constant in units of inverse Gyr

Do you remember how old our universe is in giga-years (Gyr)? If you don't, you hopefully remember that the Hubble time gives us a characteristic age, 1/$H_0$.

In [ ]:
print(1/H0)

The complete Friedmann equation for our universe includes three types of "stuff" each with their own equation of state, plus a term that can account for space-time curvature (Ryden, equation 6.6):

${\frac {H^{2}}{H_0^{2}}}= \frac{\Omega _{m,0}}{a^{3}} + \frac{\Omega _{r,0}}{a^{4}} + \Omega _{\Lambda,0} + \frac{1 - \Omega_{0}}{a^{2}}.$

Observations indicate our universe is very close to flat, but may not be exactly flat, so that last term may NOT be zero.

We would like to plot the scale factor as a function of time. Since $H(t) = \frac{\dot a^2}{a^2}$ we can re-arrange the Friedmann equation to be in terms of $a$ and $t$ and integrate it numerically:

${\dot a^{2}}= H_{0}^{2} \big( \frac{\Omega _{m,0}}{a} + \frac{\Omega _{r,0}}{a^{2}} + \Omega _{\Lambda,0}a^2 + 1 - \Omega_{0} \big)$

Below I define this function in python.

In [ ]:
def universe(a, omegaM, omegaR, omegaL):
    omega = omegaM + omegaR + omegaL
    uni = 1./np.sqrt( omegaM/a + (omegaR/(a**2.)) + omegaL*(a**2.) + (1.-omega))
    return uni/H0.value

The numerical method used below to integrate the above function is called the "trapezoid method" where the area beneath the curve is approximated by trapezoids rather than by rectangles. You doubtlessly encounter this whenever you first start to learn about numerical methods in computation. This numerical approach is defined as:

$\int^{x_2}_{x_1} f(x) dx = \frac{x_2 - x_1}{2}[f(x_1)+f(x_2)] + \frac{(x_1 - x_2)^3}{12}f''(\xi)$

where the final term is the error associated with the calculation and is not used in our integration.

In [ ]:
# Code that define the trapezoid method of integration
def trapezoid (x,fx):
    eq = 0
    for i in range(len(x)-1):
        eq= eq + (((x[i-1]-x[i])*(fx[i]+fx[i+1]))/2.)
    return eq

# Code to integrate using the Trapezoid Method
# this will calculate time from the scale factor initial values and Friedmann equation
def integrator(at,omegaM ,omegaR, omegaL):
    model_time = [0]
    model_partresult = 0
    for i in range(1, NOSteps+1):
        model_y0 = universe(at[i], omegaM, omegaR, omegaL)
        model_y1 = universe(at[i-1], omegaM, omegaR, omegaL)
        model_partresult += StepSize *(model_y0 + model_y1)/2.
        model_time.append(model_partresult)
    return np.array(model_time)

Notice how the integrator takes as an input the array of values for the scale factor, $a$, and returns an array of times matched to those scale factors. This means that every scale factor must have a unique time and vice versa (which will give us trouble later). If this is not the case, you can split your numerical integral into parts with different ranges of a and t to calculate it.

Now, all that is left is to specify the integration steps and model parameters ($\Omega_m, \Omega_r, \Omega_\Lambda$) and run the integration. We will start with the Benchmark model from Ryden's "Introduction to Cosmology".

In [ ]:
# this specifies the beginning and ending values of the scale factor, a
begin = 1e-5
end = 2
NOSteps = 1000
StepSize = (end-begin)/NOSteps
a_values = np.arange(begin,end + StepSize, StepSize)

#Benchmark Model
t_values = integrator(a_values,omega_m,omega_r,omega_de)
plt.plot(t_values, a_values)
plt.grid(True)
plt.xlabel('Time/Gyr')
plt.ylabel('Scale factor a')
plt.title("Benchmark Universe")

print(f"Generated {len(a_values)} data points")
print(f"Time range: {t_values.min():.2f} to {t_values.max():.2f} Gyr")
print(f"Scale factor range: {a_values.min():.2f} to {a_values.max():.2f}")


We want to label where we are on this plot using a dashed vertical line. First we need to find where in time the scale factor is equal to 1. Let's also make the horizontal axis logarithmic.

In [ ]:
# Calculate age of universe (time at a=1)
index_now = np.where(a_values > 1.)
time_now = t_values[index_now[0][0]]

# Remake Plot with labeled red line
plt.plot(t_values, a_values)
plt.grid(True)
plt.xlabel('Time/Gyr')
plt.ylabel('Scale factor a')
plt.title("Benchmark Universe")
plt.axvline(x=time_now, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
plt.text(time_now, 0., 'Today', fontsize=10, color='red')
plt.xscale('log')

print(f"The current age of the universe is {time_now:.2f} Gyr at scale factor a(t0) = {a_values[index_now[0][0]]:.2f}")

One problem with this simple integrator is that it breaks when the scale factor stops growing and starts to decrease. Let's try a universe that collapses back in on itself in a "Big Crunch."

In [ ]:
# this specifies the beginning and ending values of the scale factor, a
begin = 1e-5
end = 2
NOSteps = 1000
StepSize = (end-begin)/NOSteps
a_values = np.arange(begin,end + StepSize, StepSize)

#Lambda Collapse
t_crunch = integrator(a_values, 10., 0., 0.)
plt.plot(t_crunch, a_values)
plt.grid(True)
plt.xlabel('Time/Gyr')
plt.ylabel('Scale factor a')
plt.title("Crunchy Universe")

Note that python spits out a warning ``invalid value encountered in sqrt`` telling you that the right-hand side of the Friedmann equation has gone negative! Unfortunately, I didn't figure out how to get around this... but you know it looks just like the first part in reverse. So to find how long this universe lasts from Big Bang to Big Crunch, just double the time it takes to hit this mid-life crisis.

In [ ]:
# Find where integrator failed because NaNs reported
valid_mask = ~np.isnan(t_crunch)
time_crunch = 2.*t_crunch[valid_mask][-1]

print(f"This universe only exists for {time_crunch:.2f} Gyr before it ends in a crunch.")

# Question 2
Figure out how to alter the code so that the horizontal x-axis always displays time relative to today ($t-t_0$) instead of using the calculated time since the beginning of the universe.

To test this out, try plotting at least two different cosmological models and make sure that they overlap at the point ($t_0 = 0, a(t_0)=1$).

You will need to find the element in the array listing scale factors, ``a_values``, where the universe exceeds a scale factor of 1. There are many ways to do this in python, but a very useful function in this case is [``np.argmax``](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

When you have succeed, recreated Figure 5.2 from Ryden which shows the scale factor versus time for an expanding, empty universe, a flat matter-only, flat radiation-only, and flat $\Lambda$-only universes.

# Question 3

Keep experimenting with flat universes with different density values of matter that are close to 1. You should be able to create a "coasting" universe that looks like a log-function, a "ripping" universe that blows up exponentially, and a "crunching" universe that closes back in on itself.
1. Create a plot comparing all three models below.
2. What is the total density of each of these universes compared to the critical density?

# Question 4

What parameters produce a "loitering universe" where $a(t_0) = 1$ for several billion years before it explodes in size? Record your plot and combination of density parameters below. How can you increase the loitering time of your universe? Who in class can get the longest loitering universe?

# Question 5

How can you produce a "collapsing universe"? Make one that starts really large and collapses down into a "big crunch". Include your plot and density parameters below. How did you change the beginning and ending bounds for your integration?

# Question 6

What parameters produce a "big bounce" universe that doesn't originate with a big bang but instead has always existed, but decreased in size before "bouncing" into an expanding universe. Record your plot and combination of density parameters below.

# Question 7

One of the more recent speculations in cosmology is that the universe may contain a quantum field, called “quintessence,” which has a positive energy density and a negative value of the equation-of-state parameter w that varies with time. Assume, for the purposes of this problem, that the universe is spatially flat, and contains nothing but matter (w = 0), and quintessence. The equation of state of quintessence can be modeled as
$w(a) = w_0 + w_a (1+a)$, where the latest observational constraints place $w_0 =-0.7$ and $w_a = -1.0$. A negative $w_a$ means that dark energy's influence is weakening over time.

Update the code replaced $\Lambda$ with quintessence and simulate our universe. Does this change the ultimate fate of our universe?

# Question 8

Distance-Redshift relation: In this problem, you will compute distances as a function of redshift numerically. For the comoving radial distance D(z) you will need to compute numerically the integral. Plot the 3 distances (radial, angular-diameter and luminosity) as a function of redshift z for the benchmark case and for at least two cosmology variations.